In [ ]:
import ollama

In [ ]:
response = ollama.chat(model='llama3.1', messages=[
  {
    'role': 'user',
    'content': 'Is the sky be blue?, I want that you say yes or no',
  },
])
print(response['message']['content'])

In [ ]:
ll=ollama.generate(model='llama3.1', prompt='Why is the sky blue?')

In [ ]:
ll['response']

In [ ]:
def ReturnResponse(request):
    response=